In [124]:
using DataFrames
using CSV
using Query

using Combinatorics

include("../src/ColorfulFCM.jl")

Main.ColorfulFCM

In [9]:
Device = ColorfulFCM.ThermoFisherDevice

marker_dye_sheet = CSV.read("../data/使用抗体.csv", allowmissing=:none)
dye_emission_sheet = CSV.read("../data/常见染料_发射曲线.csv", allowmissing=:none)
dye_excitation_sheet = CSV.read("../data/常见染料_激发曲线.csv", allowmissing=:none);

In [121]:
readin_marker_density = [
    (:B220, 10000), 
    (:IgD,  20), 
    (:Bcl_6,4000),
    (:GL_7, 10000),
    (:FAS,  3000),
]

5-element Array{Tuple{Symbol,Int64},1}:
 (:B220, 10000)
 (:IgD, 20)    
 (:Bcl_6, 4000)
 (:GL_7, 10000)
 (:FAS, 3000)  

In [55]:
function get_dyes_candidate(df::DataFrame, markername::Symbol)
    @from item in df begin
        @where item.markers == String(markername)
        @select Symbol(item.dyes)
        @collect
    end
end

function get_dye_wave_score(df::DataFrame, dyename::Symbol, loc::Int64, scale::Int64=0)
    waverange = range(loc-scale, stop=loc+scale, step=1)
    score_array = @from item in df begin
        @where item.Wavelength in waverange
        @select item[dyename]
        @collect
    end
    _score = mean(score_array)
    return _score > 0 ? _score : 0.0
end

$$ \frac{\alpha_{ex}}{100} \cdot \langle \alpha_{em} \rangle \cdot \alpha_L$$

In [122]:
sparse_interaction = DataFrame(
    laser=Vector{Symbol}(undef, 0),
    lens_mu = Vector{Int64}(undef, 0),
    lens_sig = Vector{Int64}(undef, 0),
    marker = Vector{Symbol}(undef, 0),
    dye = Vector{Symbol}(undef, 0),
    score = Vector{Float64}(undef, 0)
)

for (laser_name, laser_lambda) in Device.lasers, (marker_name, marker_density) in readin_marker_density
    dye_candidate = get_dyes_candidate(marker_dye_sheet, marker_name)
    for (lens_mu, lens_sig) in Device.lens[laser_name], dye_name in dye_candidate
        excitation_score = get_dye_wave_score(dye_excitation_sheet, dye_name, laser_lambda)
        emission_score = get_dye_wave_score(dye_emission_sheet, dye_name, lens_mu, lens_sig)
        _score = excitation_score * emission_score * log10(marker_density)
        push!(sparse_interaction, [laser_name, lens_mu, lens_sig, marker_name, dye_name, _score])
    end
end

In [126]:
?permutations

search: permutations multiset_permutations



```
permutations(a)
```

Generate all permutations of an indexable object `a` in lexicographic order. Because the number of permutations can be very large, this function returns an iterator object. Use `collect(permutations(a))` to get an array of all permutations.

---

```
permutations(a, t)
```

Generate all size `t` permutations of an indexable object `a`.


In [128]:
permutations(readin_marker_density)

960

In [133]:
head(sparse_interaction)

,laser,lens_mu,lens_sig,marker,dye,score
,Symbol,Int64,Int64,Symbol,Symbol,Float64
1,red,660,20,B220,APC,21199.6
2,red,660,20,B220,APC_Cy7,2586.87
3,red,660,20,B220,BV605,0.0
4,red,660,20,B220,FITC,0.0
5,red,660,20,B220,PacificBlue,0.0
6,red,660,20,B220,PE,0.0


In [137]:
solution_order = permutations(readin_marker_density)
solution = Vector{DataFrame}(undef, 0)
for marker_order in solution_order
    epoch_sheet = sparse_interaction
    
    possible_result = DataFrame(
        laser=Vector{Symbol}(undef, 0),
        lens_mu = Vector{Int64}(undef, 0),
        lens_sig = Vector{Int64}(undef, 0),
        marker = Vector{Symbol}(undef, 0),
        dye = Vector{Symbol}(undef, 0),
        score = Vector{Float64}(undef, 0)
    )
    
    try
        for (each_marker, _) in marker_order
            thismarker_sheet = epoch_sheet[epoch_sheet.marker .== each_marker, :]
            j, ridx = findmax(thismarker_sheet.score)
            _choice = thismarker_sheet[ridx, :]
            push!(possible_result, convert(Array, _choice))

            epoch_sheet = epoch_sheet[(epoch_sheet.lens_mu .!= _choice.lens_mu).&(epoch_sheet.marker .!= _choice.marker).&(epoch_sheet.dye .!= _choice.dye), :]
        end
    catch e
        continue
    end
    
    push!(solution, possible_result)

end

┌ Warning: Deprecated syntax `implicit assignment to global variable `_``.
│ Use `global _` instead.
└ @ nothing none:0
┌ Warning: Loop variable `_` around In[137]:16 overwrites a variable in an enclosing scope. In the future the variable will be local to the loop instead.
└ @ nothing In[137]:16


In [143]:
(value, optimal_idx) = [sum(item.score) for item in solution] |> findmax
solution[optimal_idx]

,laser,lens_mu,lens_sig,marker,dye,score
,Symbol,Int64,Int64,Symbol,Symbol,Float64
1,blue,710,50,B220,Percp_Cy5_5,21638.9
2,blue,530,30,IgD,FITC,6577.36
3,red,660,20,FAS,APC,18428.4
4,red,730,45,Bcl_6,Alexa647,6675.3
5,blue,575,26,GL_7,PE,10308.7
